In [1]:
from gen_embeddings import *

In [2]:
files = files = sorted([os.path.abspath(os.path.join(dirp, f)) for dirp, _, fn in
             os.walk('/Users/garrettsmith/Google Drive/UniPotsdam/Research/Features/dependency_embeddings/data/ParsedBrownCorpus/') for f in fn if f.endswith('.txt')])

In [3]:
deps = read_standford(files)

In [4]:
pmi_dict = make_pmi_dict(deps, positive=True)

In [5]:
sdf = to_sparse_df(pmi_dict)

## Test case
- *The canoe by the cabin was damaged* $\rightarrow$ nsubj(damaged, canoe), nsubj(damaged, cabin)
- *The canoe by the kayak was damaged* $\rightarrow$ nsubj(damaged, canoe), nsubj(damaged, kayak)

In [9]:
pairs = [('damaged', 'nsubj')]
nouns = ['canoe', 'cabin', 'boat']
damaged = lex_spec_feats(pairs, deps, sdf)

Calculating lexically specific dependent features...
Working on pair damaged-nsubj.
Done.


In [12]:
sims = calc_similarity(nouns, ['damaged-nsubj'], sdf, damaged)
sims

,damaged-nsubj
canoe,0.418307
cabin,0.419959
boat,0.509556


In [16]:
cancab = sims.loc['canoe'] - sims.loc['cabin']
canboat = sims.loc['canoe'] - sims.loc['boat']
print('cos(damaged, canoe) - cos(damaged, cabin) = {}'.format(cancab))
print('cos(damaged, canoe) - cos(damaged, boat) = {}'.format(canboat))
print('"Boat" cost relative to "cabin"'.format(cancab - canboat))

cos(damaged, canoe) - cos(damaged, cabin) = damaged-nsubj   -0.001651
dtype: float64
cos(damaged, canoe) - cos(damaged, boat) = damaged-nsubj   -0.091249
dtype: float64
"Boat" cost relative to "cabin"


## Trying out Cunnings & Sturt's (2018) materals

- obj(shattered, letter) - obj(shattered, cup)
- obj(shattered, letter) - obj(shattered, tie)

In [23]:
pairs = [('shatter', 'obj')]
nouns = ['letter', 'cup', 'tie']
shattered = lex_spec_feats(pairs, deps, sdf)

Calculating lexically specific dependent features...
Working on pair shatter-obj.
Done.


In [25]:
sims = calc_similarity(nouns, ['shatter-obj'], sdf, damaged)
sims

AssertionError: Not all words in word dataframe.

In [26]:
import pandas as pd

In [48]:
cs = pd.read_csv('/Users/garrettsmith/Google Drive/UniPotsdam/Research/Features/CunningsSturtMaterials.csv',
                skipinitialspace=True)

In [49]:
cs.head()

,item,tplaus,dplaus,verb,target,distractor
0,1,plaus,plaus,shattered,plate,cup
1,1,plaus,implaus,shattered,plate,tie
2,1,implaus,plaus,shattered,letter,cup
3,1,implaus,implaus,shattered,letter,tie
4,2,plaus,plaus,moved,tent,chair


In [50]:
implaus = cs.loc[cs.tplaus == 'implaus']
implaus.head()

,item,tplaus,dplaus,verb,target,distractor
2,1,implaus,plaus,shattered,letter,cup
3,1,implaus,implaus,shattered,letter,tie
6,2,implaus,plaus,moved,tree,chair
7,2,implaus,implaus,moved,tree,river
10,3,implaus,plaus,planted,tent,flowers


In [51]:
for v in set(implaus.verb):
    try:
        sdf.loc[v]
    except:
        print(v)

In [52]:
for t in set(implaus.target):
    try:
        sdf.loc[t]
    except:
        print(t)

laptop
teapot
cutlery


In [53]:
for d in set(implaus.distractor):
    try:
        sdf.loc[d]
    except:
        print(d)

bike


In [66]:
implaus.loc[implaus.target.isin(['laptop', 'teapot', 'cutlery']) | implaus.distractor.isin(['bike'])]

,item,tplaus,dplaus,verb,target,distractor
14,4,implaus,plaus,cooked,cutlery,food
15,4,implaus,implaus,cooked,cutlery,sink
38,10,implaus,plaus,wrote,laptop,report
39,10,implaus,implaus,wrote,laptop,office
82,21,implaus,plaus,rode,cardboard,bike
114,29,implaus,plaus,toasted,teapot,bread
115,29,implaus,implaus,toasted,teapot,plate
